## 准备环境

In [1]:
import requests
import re
import time

## 爬取电影数据

In [3]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0'
}

x = range(0, 125, 25)

for i in x:
    # 请求排行榜页面
    html = requests.get("https://movie.douban.com/top250?start=" + str(i), headers=headers)
    # 防止请求过于频繁
    time.sleep(0.01)
    # 将获取的内容采用utf8解码
    cont = html.content.decode('utf8')
    # 使用正则表达式获取电影的详细页面链接
    urlList = re.findall('<a href="https://movie.douban.com/subject/\d*?/" class="">', cont)
    # 排行榜每一页都有25个电影，于是匹配到了25个链接，逐个对访问进行请求
    for j in range(len(urlList)):
        # 获取标签中的url
        url = urlList[j].replace('<a href="', "").replace('" class="">', "")
        # 将获取的内容采用utf8解码
        content = requests.get(url, headers=headers).content.decode('utf8')
        # 采用数字作为文件名
        print(f'i = {i}, j = {j}')
        film_name = i + j
        # 写入文件
        with open('contents/' + str(film_name) + '.txt', mode='w', encoding='utf8') as f:
            f.write(content)

结点获取

In [22]:
import re
import pandas

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'
}


def node_save(attrCont, tag, attr, label):
    ID = []
    for i in range(len(attrCont)):
        ID.append(tag * 10000 + i)
    data = {'ID': ID, attr: attrCont, 'LABEL': label}
    dataframe = pandas.DataFrame(data)
    dataframe.to_csv('details/' + attr + '.csv', index=False, sep=',', encoding="utf_8_sig")


def save(contents):
    # save movie nodes
    film_name = re.findall('<title>.*?/title>', contents)[0]
    film_name = film_name.lstrip("<title>").rstrip("(豆瓣)</title>").replace(" ", "")
    film_names.append(film_name)

    # save director nodes
    director_cont = re.findall('"director":.*?]', contents)[0]
    director_cont = re.findall('"name": ".*?"', director_cont)
    for i in range(len(director_cont)):
        directors.append(director_cont[i].lstrip('"name": "').rstrip('"'))

    # save actors nodes
    actor_cont = re.findall('"actor":.*?]', contents)[0]
    actor_cont = re.findall('"name": ".*?"', actor_cont)
    for i in range(len(actor_cont)):
        actors.append(actor_cont[i].lstrip('"name": "').rstrip('"'))

    # save type
    type_cont = re.findall('<span property="v:genre">.*?</span>', contents)
    for i in range(len(type_cont)):
        types.append(type_cont[i].lstrip('<span property="v:genre">').rstrip('</span>'))


film_names = []
actors = []
directors = []
types = []
for i in range(125):
    with open('contents/' + str(i) + '.txt', mode='r', encoding='utf8') as f:
        contents = f.read()
    save(contents.replace("\n", ""))  # 这里需要把读出来的数据换行符去掉

# 去重
actors = list(set(actors))
directors = list(set(directors))
types = list(set(types))
# 保存
node_save(film_names, 0, 'film_name', 'movie')
node_save(directors, 1, 'director', 'person')
node_save(actors, 2, 'actor', 'person')
node_save(types, 3, "type", "type")
print('Done')

Done


结点关系的获取

In [24]:
def getID(name, nameValue):
    df = pandas.read_csv('details/' + name + '.csv')
    for j in range(len(df[name])):
        if nameValue == df[name][j]:
            return df['ID'][j]


acted_in_data = pandas.DataFrame()
directed_data = pandas.DataFrame()
cooperation_data = pandas.DataFrame()
belong_to_data = pandas.DataFrame()


def save_relation(start_id, end_id, relation):
    dataframe = pandas.DataFrame({':START_ID': start_id, ':END_ID': end_id, ':relation': relation, ':TYPE': relation})
    dataframe.to_csv('details/' + relation + '.csv', index=False, sep=',', encoding="utf_8_sig")


def save_acted_in(content):
    # 获取当前电影对应ID
    film_name = re.findall('<title>.*?/title>', content)[0]
    film_name = film_name.lstrip("<title>").rstrip("(豆瓣)</title>").replace(" ", "")  # 电影名字每页只有一个
    filmNameID = getID('film_name', film_name)

    # 获取当前电影的演员和对应ID
    actor_cont = re.findall('"actor":.*?]', content)[0]
    actor_cont = re.findall('"name": ".*?"', actor_cont)
    for i in range(len(actor_cont)):  # 演员每页可能多个（通常都多个)
        actor = actor_cont[i].lstrip('name": "').rstrip('"')
        start_id.append(filmNameID)
        end_id.append(getID('actor', actor))  # 查找演员名字对应ID


def save_directed(contnet):
    # 获取当前电影对应ID
    film_name = re.findall('<title>.*?/title>', content)[0]
    film_name = film_name.lstrip("<title>").rstrip("(豆瓣)</title>").replace(" ", "")
    filmNameID = getID('film_name', film_name)

    #
    director_cont = re.findall('"director":.*?]', content)[0]
    director_cont = re.findall('"name": ".*?"', director_cont)
    for i in range(len(director_cont)):
        director = director_cont[i].lstrip('"name": "').rstrip('"')
        start_id.append(filmNameID)
        end_id.append(getID('director', director))


def save_belongto(content):
    # 获取当前电影对应ID
    film_name = re.findall('<title>.*?/title>', content)[0]
    film_name = film_name.lstrip("<title>").rstrip("(豆瓣)</title>").replace(" ", "")
    filmNameID = getID('film_name', film_name)

    #
    type_cont = re.findall('<span property="v:genre">.*?</span>', content)
    for i in range(len(type_cont)):
        type = type_cont[i].lstrip('<span property="v:genre">').rstrip('</span>')
        start_id.append(filmNameID)
        end_id.append(getID('type', type))


def save_cooperation(content):
    # 获取当前电影的演员和对应ID
    actor_cont = re.findall('"actor":.*?]', content)[0]
    actor_cont = re.findall('"name": ".*?"', actor_cont)

    #
    director_cont = re.findall('"director":.*?]', content)[0]
    director_cont = re.findall('"name": ".*?"', director_cont)

    for i in range(len(actor_cont)):
        actor = actor_cont[i].lstrip('name": "').rstrip('"')
        for j in range(len(director_cont)):
            director = director_cont[j].lstrip('"name": "').rstrip('"')
            start_id.append(getID('actor', actor))
            end_id.append(getID('director', director))


# 用来存放关系节点ID的列表
start_id = []
end_id = []

# 循环查找每个页面（即contents文件夹中下载下来的页面），找出对应关系(acted_in)
for i in range(125):
    with open('contents/' + str(i) + '.txt', mode='r', encoding='utf8') as f:
        content = f.read().replace('\n', "")  # 要去掉换行符
    save_acted_in(content)
save_relation(start_id, end_id, 'acted_in')
print('[+] save acted_in finished!!!!!!!!!!!!!!!!!')

start_id.clear()
end_id.clear()
# 循环查找每个页面（即contents文件夹中下载下来的页面），找出对应关系(directed)
for i in range(125):
    with open('contents/' + str(i) + '.txt', mode='r', encoding='utf8') as f:
        content = f.read().replace('\n', "")  # 要去掉换行符
    save_directed(content)
save_relation(start_id, end_id, 'directed')
print('[+] save directed finished!!!!!!!!!!!!!!!!!')

start_id.clear()
end_id.clear()
# 循环查找每个页面（即contents文件夹中下载下来的页面），找出对应关系(belong_to)
for i in range(125):
    with open('contents/' + str(i) + '.txt', mode='r', encoding='utf8') as f:
        content = f.read().replace('\n', "")  # 要去掉换行符
    save_belongto(content)
save_relation(start_id, end_id, 'belong_to')
print('[+] save belong_to finished!!!!!!!!!!!!!!!!!')

start_id.clear()
end_id.clear()
# 循环查找每个页面（即contents文件夹中下载下来的页面），找出对应关系(cooperation)
for i in range(125):
    with open('contents/' + str(i) + '.txt', mode='r', encoding='utf8') as f:
        content = f.read().replace('\n', "")  # 要去掉换行符
    save_cooperation(content)
save_relation(start_id, end_id, 'cooperation')
print('[+] save cooperation finished!!!!!!!!!!!!!!!!!')

[+] save acted_in finished!!!!!!!!!!!!!!!!!
[+] save directed finished!!!!!!!!!!!!!!!!!
[+] save belong_to finished!!!!!!!!!!!!!!!!!
[+] save cooperation finished!!!!!!!!!!!!!!!!!
